In [16]:
import torch
import torch.optim as optim
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

In [6]:
mean_n = (0.5, 0.5, 0.5)
std_n = (0.5, 0.5, 0.5)

transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(227),
    transforms.ToTensor(),
    transforms.Normalize(mean_n, std_n) # n채널에 대해 mean, std 정규화
])

In [7]:
trainset = torchvision.datasets.STL10(root='./data', split='train', download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=32, shuffle=True)

testset = torchvision.datasets.STL10(root='./data', split='test', download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=32, shuffle=False)


Extracting ./data/stl10_binary.tar.gz to ./data
Files already downloaded and verified


In [13]:
"""
LRN (LocalResponseNorm) Arguments
# size: amount of neighbouring channels used for normalization
# alpha: multiplicative factor. Default: 0.0001
# beta: exponent. Default: 0.75
# k: additive factor. Default: 1
"""

class AlexNetSingleGPU(nn.Module):
  def __init__(self, num_classes:int = 1000, init_weights:bool = True):
    super(AlexNetSingleGPU, self).__init__()
    self.feature_extract = nn.Sequential(
        # 227*227*3 -> 55*55*96 // ((227+2*0-11)/4) + 1 = 216/4 + 1 = 54 +1 = 55
        nn.Conv2d(in_channels=3, out_channels=96, kernel_size=11, padding=0, stride=4),
        nn.ReLU(inplace=True),
        nn.LocalResponseNorm(size=5, k=2),
        # 55*55*96 -> 27*27*96
        nn.MaxPool2d(kernel_size=3, stride=2), 

        # 27*27*96 -> 27*27*256 // ((27+2*2-5)/1) + 1 = 26 + 1 = 27
        nn.Conv2d(in_channels=96, out_channels=256, kernel_size=5, padding=2, stride=1),
        nn.ReLU(inplace=True),
        nn.LocalResponseNorm(size=5, k=2),
        # 27*27*256 -> 13*13*256
        nn.MaxPool2d(kernel_size=3, stride=2),

        # 13*13*256 -> 13*13*384 // ((13+2*1-3)/1) + 1 = 12 + 1 = 13
        nn.Conv2d(in_channels=256, out_channels=384, kernel_size=3, padding=1, stride=1),
        nn.ReLU(inplace=True),
        nn.LocalResponseNorm(size=5, k=2),

        # 13*13*384 -> 13*13*384 // ((13+2*1-3)/1) + 1 = 12 + 1 = 13
        nn.Conv2d(in_channels=384, out_channels=384, kernel_size=3, padding=1, stride=1),
        nn.ReLU(inplace=True),
        nn.LocalResponseNorm(size=5, k=2),

        # 13*13*384 -> 13*13*256 // ((13+2*1-3)/1) + 1 = 12 + 1 = 13
        nn.Conv2d(in_channels=384, out_channels=256, kernel_size=3, padding=1, stride=1),
        nn.ReLU(inplace=True),
        nn.LocalResponseNorm(size=5, k=2),
        # 13*13*256 -> 6*6*256
        nn.MaxPool2d(kernel_size=3, stride=2),
    )

    self.fully_connected = nn.Sequential(
        nn.Linear(6*6*256, 4096),
        nn.ReLU(inplace=True),
        nn.Dropout(p=0.5),

        nn.Linear(4096, 4096),
        nn.ReLU(inplace=True),
        nn.Dropout(p=0.5),

        nn.Linear(4096, num_classes)
    )

  def forward(self, x: torch.Tensor):
    x = self.feature_extract(x)
    x = torch.flatten(x, 1)
    x = self.fully_connected(x)

    return x

In [18]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
alexnet = AlexNetSingleGPU(1000)
alexnet = alexnet.to(device)

In [17]:
criterion = nn.CrossEntropyLoss().cuda()
optimizer = optim.SGD(alexnet.parameters(), lr=0.01, momentum=0.9, weight_decay=0.0005)